In [438]:
from reportlab.pdfgen import canvas
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.cidfonts import UnicodeCIDFont
from reportlab.lib.pagesizes import A4, A5, portrait, landscape
from reportlab.lib.units import mm
from reportlab.platypus import Table, TableStyle
from reportlab.lib import colors
import numpy as np
import cv2
import matplotlib.pyplot as plt
from IPython.display import display_pdf

In [433]:
COLOR_DIC = {'0 White': [242, 243, 242],
 '1 Tan': [111, 160, 176],
 '5 Nougat': [104, 145, 208],
 '6 Red': [9, 26, 201],
 '8 Yellow': [55, 205, 242],
 '9 Black': [29, 19, 5],
 '16 Md,Orange': [11, 167, 255],
 '17 Orange': [24, 138, 254],
 '21 Magenta': [118, 31, 144],
 '23 Md,Nougat': [42, 112, 204],
 '28 Dark Red': [15, 14, 114],
 '29 Bt,Lt Orange': [61, 187, 248],
 '31 Light Bluish Gray': [169, 165, 160],
 '32 Dark Bluish Gray': [104, 110, 108],
 '33 Very Lt, Bluish Gray': [224, 227, 230],
 '34 Bt, Lt Blue': [233, 195, 159],
 '44 Coral': [80, 127, 255]}

In [434]:
COLOR_DIC_ALL = {'0 White': [242, 243, 242], '1 Tan': [111, 160, 176], '2 Light Green': [168, 217, 173], '3 Maersk Blue': [195, 146, 53], '4 Pink': [255, 217, 171],
             '5 Nougat': [104, 145, 208], '6 Red': [9, 26, 201], '7 Blue': [191, 85, 0], '8 Yellow': [55, 205, 242], '9 Black': [29, 19, 5],
             '10 Green': [65, 120, 35], '11 Md,Green': [117, 196, 127], '12 Bt,Green': [65, 171, 88], '13 Dark Orange': [0, 85, 169],
             '14 Light Violet': [226, 202, 201], '15 Md,Blue': [219, 147, 90], '16 Md,Orange': [11, 167, 255], '17 Orange': [24, 138, 254],
             '18 Blue-Violet': [202, 116, 104], '19 Light Turquoise': [175, 165, 85], '20 Lime': [11, 233, 187], '21 Magenta': [118, 31, 144],
             '22 Sand Blue': [161, 116, 96], '23 Md,Nougat': [42, 112, 204], '24 Dark Tan': [115, 138, 149], '25 Dark Blue': [99, 52, 10],
             '26 Dark Green': [50, 70, 24], '27 Sand Green': [172, 188, 160], '28 Dark Red': [15, 14, 114], '29 Bt,Lt Orange': [61, 187, 248],
             '30 Reddish Brown': [18, 42, 88], '31 Light Bluish Gray': [169, 165, 160], '32 Dark Bluish Gray': [104, 110, 108],
             '33 Very Lt, Bluish Gray': [224, 227, 230], '34 Bt, Lt Blue': [233, 195, 159], '35 Dark Pink': [160, 112, 200],
             '36 Bright Pink': [200, 173, 228], '37 Bt,Lt Yellow': [58, 240, 255], '38 Dark Purple': [145, 54, 63], '39 Light Nougat': [179, 215, 246],
             '40 Dark Brown': [0, 33, 53], '41 Light Aqua': [234, 242, 211], '42 Md,Lavender': [185, 110, 160], '43 Lavender': [222, 164, 205],
             '44 Coral': [80, 127, 255]
            }

In [448]:
"""
説明書PDFを作成する関数
color_map: LEGOで作成する最終的なカラー画像
depth_map: LEGOで作成する最終的な深度画像
min_color_num: 関数'change_coler'で返される
COLOR_DIC: 使用する色の辞書型配列
"""
def creat_instruction(color_map, depth_map, min_color_num, COLOR_DIC):
    table_len = 15
    table_len_x, table_len_y = table_len, table_len
    
    Height, Width = depth_map.shape[:2]
    #print(Height, Width)
    Height_num = Height//table_len_y
    Width_num = Width//table_len_x
    #print(Height_num, Width_num)
    
    # 縦型A4のCanvasを準備
    cv = canvas.Canvas('Instruction.pdf', pagesize=portrait(A4))
    cv.setTitle('Instruction')
    # フォント登録
    pdfmetrics.registerFont(UnicodeCIDFont('HeiseiKakuGo-W5'))
    #完成図の挿入
    image_path = './Image_inst/color_map.png'
    cv2.imwrite(image_path, color_map)
    cv.drawImage(image=image_path, x=10*mm, y=50*mm, width=190*mm, height=(Height*190/Width)*mm, mask='auto')
    #改ページ
    cv.showPage()
    
    Color_list = []
    for i, key in enumerate(COLOR_DIC):
        Color_list.append(COLOR_DIC[key])
    Color_len = len(Color_list)
    Color_len_syou = Color_len//table_len
    color_list_data = [[ j*table_len+i+1 for i in range(table_len)] for j in range(Color_len_syou)]
    if Color_len%table_len!=0:
        color_list_data += [[table_len*Color_len_syou+i+1 for i in range(Color_len%table_len)]+[" "]*(table_len-Color_len%table_len)]
    table_color = Table(color_list_data, colWidths=12*mm, rowHeights=12*mm)
    table_color.setStyle(TableStyle([
        ('FONT', (0, 0), (-1, -1), 'HeiseiKakuGo-W5', 15), # フォント
        ('GRID', (0, 0), (-1, -1), 1, colors.black),       # 罫線
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER')
    ]))
    for y in range(len(color_list_data)):
        for x in range(table_len):
            if y*table_len+x < Color_len:
                b,g,r = Color_list[y*table_len+x]
                bgr = int(b)+int(g)+int(r)
                if bgr >= 382:
                    wordColor = colors.black
                else:
                    wordColor = colors.white
                table_color.setStyle(TableStyle([
                    ('TEXTCOLOR', (x,y), (x,y), wordColor),
                    ('BACKGROUND', (x,y), (x,y), colors.Color(1.0/255*r,1.0/255*g,1.0/255*b, 1))
                ]))
            else:
                table_color.setStyle(TableStyle([
                    ('SPAN', (x, y) , (-1, y))
                ]))
                break
    
    for h in range(Height_num+1):
        table_len_x, table_len_y = table_len, table_len
        if h==Height_num:
            if Height%table_len!=0:
                table_len_y = Height % table_len
            else:
                continue
        for w in range(Width_num+1):
            if w==Width_num:
                if Width%table_len!=0:
                    table_len_x = Width % table_len
                else:
                    continue
            #data = [[" " for _ in range(table_len)] for __ in range(table_len)]
            data_color = min_color_num[table_len*h:table_len*h+table_len_y, table_len*w:table_len*w+table_len_x].tolist()
            data_depth = depth_map[table_len*h:table_len*h+table_len_y, table_len*w:table_len*w+table_len_x].tolist()
            data = list(map(lambda x,y: list(map(lambda a,b: str(a+1)+str("×")+str(b), x,y)), data_color, data_depth))
            table = Table(data, colWidths=12*mm, rowHeights=12*mm)
            # tableの装飾
            table.setStyle(TableStyle([                              
                ('FONT', (0, 0), (-1, -1), 'HeiseiKakuGo-W5', 8), # フォント
                ('GRID', (0, 0), (-1, -1), 1, colors.black),       # 罫線
                ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
                ('ALIGN', (0, 0), (-1, -1), 'CENTER')
            ]))
            for x in range(table_len_x):
                for y in range(table_len_y):
                    b,g,r = color_map[table_len*h+y, table_len*w+x]
                    bgr = int(b)+int(g)+int(r)
                    if bgr >= 382:
                        wordColor = colors.black
                    else:
                        wordColor = colors.white
                    table.setStyle(TableStyle([
                        ('TEXTCOLOR', (x,y), (x,y), wordColor),
                        ('BACKGROUND', (x,y), (x,y), colors.Color(1.0/255*r,1.0/255*g,1.0/255*b, 1))
                    ]))
            #Artのカラーテーブル
            table.wrapOn(cv, 15*mm, 10*mm) # table位置
            table.drawOn(cv, 15*mm, 10*mm)
            #使用する色のカラーテーブル
            table_color.wrapOn(cv, 15*mm, 195*mm) # table位置
            table_color.drawOn(cv, 15*mm, 195*mm)
            #画像の挿入
            image_path = f'./Image_inst/{h}_{w}.png'
            sample_image = np.full_like(color_map,255)
            sample_image = cv2.copyMakeBorder(sample_image, 1, 1, 1, 1, cv2.BORDER_CONSTANT, (0,0,0))
            sample_image[table_len*h+1:table_len*h+table_len_y+1, table_len*w+1:table_len*w+table_len_x+1] = color_map[table_len*h:table_len*h+table_len_y, table_len*w:table_len*w+table_len_x]
            cv2.imwrite(image_path, sample_image)
            cv.drawImage(image=image_path, x=15*mm, y=240*mm, width=(Width*50/Height)*mm, height=50*mm, mask='auto')
            #改ページ
            cv.showPage()
            """
            plt.imshow(color_map[table_len*h:table_len*h+table_len, table_len*w:table_len*w+table_len,::-1])
            plt.show()
            """
    # 保存
    cv.save()

In [449]:
color_map = cv2.imread("./rgb_map.png")
depth_map = cv2.imread("./depth_map.png", 0)
min_color_num = cv2.imread("./min_color_num.png", 0)

In [450]:
creat_instruction(color_map, depth_map, min_color_num, COLOR_DIC)

In [451]:
filename = "./Instruction.pdf"
with open(filename,"rb") as f:
    display_pdf(f.read(),raw=True)